<a href="https://colab.research.google.com/github/hr7git/Data-Analysis-with-Open-Source/blob/main/15%EA%B0%95%20sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 15강 시계열 데이터 분석 : 주가 데이터 활용

### 목표

- S&P500 지수를 중심으로 주가 데이터를 수집
- 다양한 시계열 분석 기법을 적용하여 의미 있는 인사이트를 도출

> ⚠️**주의**⚠️ : 이번 장에서 다루는 내용은 순전히 교육 및 실습 목적으로 주가 데이터를 활용하는 것이며, 실제 투자 결정을 위한 가이드로 사용할 수 없다. 시계열 분석 기법은 과거 데이터의 패턴을 학습하는 데 유용하나, 금융 시장의 불확실성과 예측 불가능한 사건들로 인해 실제 투자에 적용할 경우 상당한 위험이 따를 수 있다. 어떤 분석 방법도 미래 주가를 완벽히 예측할 수 없으며, 본 내용은 데이터 분석 기술을 배우는 학습 도구로만 활용하길 권장한다.

### 분석 프로세스 개요

1. 데이터 수집
  - yfinance를 이용한 데이터 수집
  - 데이터 구조 이해
2. 금융 시계열 분석
  - 기술적 지표 계산 (이동평균, RSI)
  - 거래 신호 생성 및 거래 전략 구성
  - 백테스팅
3. Prophet 기반 예측
  - 단기 주가 예측 모델 학습 및 평가
4. GBRT 기반 예측
  - 특성 변수 설계 및 예측 모델 학습
  - 예측 기반 투자 전략 및 백테스팅

In [ ]:
# 한글 처리를 위한 matplotlib 설정 (1)

!sudo apt-get install -y fonts-nanum
!sudo fc-cache –fv
!rm ~/.cache/matplotlib -rf

- 런타임 -> 세션 다시 시작

In [ ]:
# 한글 처리를 위한 matplotlib 설정 (2)

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

# 1. 데이터 수집

## 15-1 yfinance를 이용한 S&P 500 데이터 다운로드

In [ ]:
# yfinance 임포트
import datetime

symbol = 'S&P500'
ticker = '^GSPC'

## 데이터 수집 날짜 설정 (2014년 1월 1일) ~ (2019년 12월 31일)
start_date = datetime.datetime(2014, 1, 1)
end_date = datetime.datetime(2019, 12, 31)

## yfinance를 사용하여 S&P 500 (^GSPC) 데이터를 지정된 기간 동안 다운로드

## MultiIndex 컬럼에서 두 번째 레벨(보통 'Adj Close' 등)을 제거하여 컬럼 이름을 단순화
df.columns = df.columns.droplevel(1)

## 다운로드된 DataFrame의 기술 통계량 출력
df.describe()

## 15-2 S&P 500 데이터 시각화 (훈련/테스트 분할)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.grid(True)
plt.title('S&P500')
## 2018년까지의 데이터를 훈련 데이터로 시각화
plt.plot(df[:'2018'].index, df[:'2018']['Close'], label='Train Data')
## 2018년 이후 데이터를 훈련 데이터로 시각화
plt.plot(df['2018':].index, df['2018':]['Close'], label='Test Data')
plt.legend()
plt.show()

# 2. 금융 시계열 분석

# new session

test coding

In [ ]:
!pip install yfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor

# 1. 데이터 수집 (충분한 학습을 위해 5년치 데이터를 가져옵니다)
ticker = "^GSPC"  # S&P 500 지수
df = yf.download(ticker, start="2019-01-01", end="2024-01-01")

# 2. 기술적 지표 생성 (수학적 보조지표)
# 이동평균선: 데이터의 노이즈를 제거하는 Low-pass Filter 역할
df['MA20'] = df['Close'].rolling(window=20).mean()
df['MA60'] = df['Close'].rolling(window=60).mean()
# 변동성: 주가 움직임의 표준편차
df['Volatility'] = df['Close'].rolling(window=20).std()

# 3. 매수/매도 신호 생성 (골든/데드크로스 로직)
df['Signal'] = 0
df.loc[df['MA20'] > df['MA60'], 'Signal'] = 1   # 매수 신호
df.loc[df['MA20'] < df['MA60'], 'Signal'] = -1  # 매도 신호

# 4. 머신러닝을 위한 데이터 정제
# 내일의 종가를 타겟(y)으로 설정 (Shift 연산)
df['Target'] = df['Close'].shift(-1)
df = df.dropna()  # 지표 계산으로 발생한 결측치 제거

# 독립변수(X)와 종속변수(y) 분리
features = ['Close', 'MA20', 'MA60', 'Volatility', 'Signal']
X = df[features]
y = df['Target']

# 시계열 데이터이므로 순서를 유지하며 학습/테스트 데이터 분리 (8:2)
split = int(len(df) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 5. Gradient Boosting 모델 학습 (GBRT)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# 6. 예측 실행
predictions = model.predict(X_test)
result_df = pd.DataFrame({'Actual': y_test.values.flatten(),
                          'Predicted': predictions.flatten()},
                         index=y_test.index)

# 7. 결과 시각화 (정의되지 않았던 함수 부분을 통합함)
plt.figure(figsize=(15, 8))
plt.plot(result_df['Actual'], label='Actual S&P 500', color='royalblue', linewidth=2)
plt.plot(result_df['Predicted'], label='AI Prediction', color='darkorange', linestyle='--', linewidth=2)

# 매수/매도 신호 표시 (실제 주가 위에 신호 표기)
buy_signals = X_test[X_test['Signal'] == 1].index
plt.scatter(buy_signals, y_test.loc[buy_signals], marker='^', color='green', label='Buy Signal', s=100)

plt.title(f'S&P 500 Stock Price Prediction using GBRT (Test Set)', fontsize=15)
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 모델 평가 지표 출력
from sklearn.metrics import mean_absolute_error, r2_score
print(f"평균 절대 오차 (MAE): {mean_absolute_error(y_test, predictions):.2f}")
print(f"결정 계수 (R2 Score): {r2_score(y_test, predictions):.4f}")